# CNN Model

In [1]:
# Import the necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Set up the data generators for training and validation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the image data
image_dir = './FMD/image/'
class_names = os.listdir(image_dir)

X = []
y = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join(image_dir, class_name)
    for image_path in os.listdir(class_dir):
        image = keras.preprocessing.image.load_img(os.path.join(class_dir, image_path), target_size=(224, 224))
        image_array = keras.preprocessing.image.img_to_array(image)
        X.append(image_array)
        y.append(i)

In [3]:
# Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [13]:
# Count the total number of images 
total_images = len(X)
print("Total number of images:", total_images)

# Determine the image size and type
image_size = X[0].shape
image_type = type(X[1][0][0][0])
print("Image size:", image_size)
print("Image type:", image_type)

Total number of images: 1000
Image size: (224, 224, 3)
Image type: <class 'numpy.float32'>


In [5]:
# Check the shape of the data
print("X shape:", X.shape)
print("y shape:", y.shape)

# Check the number of classes
num_classes = len(class_names)
print("Number of classes:", num_classes)

# Check the distribution of classes
class_counts = np.bincount(y)
print("Class counts:", class_counts)

# Check the class distribution as percentages
class_percentages = class_counts / len(y) * 100
print("Class percentages:", class_percentages)

X shape: (1000, 224, 224, 3)
y shape: (1000,)
Number of classes: 10
Class counts: [100 100 100 100 100 100 100 100 100 100]
Class percentages: [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Perform one-hot encoding on the labels
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
# Define the CNN model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [10]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

Epoch 1/20
7/7 [==============================] - 28s 4s/step - loss: 551.9333 - accuracy: 0.0862 - val_loss: 18.0202 - val_accuracy: 0.0750
Epoch 2/20
7/7 [==============================] - 29s 4s/step - loss: 5.0717 - accuracy: 0.1500 - val_loss: 2.2647 - val_accuracy: 0.1500
Epoch 3/20
7/7 [==============================] - 27s 4s/step - loss: 1.9947 - accuracy: 0.3762 - val_loss: 2.4851 - val_accuracy: 0.1800
Epoch 4/20
7/7 [==============================] - 26s 4s/step - loss: 1.5846 - accuracy: 0.5188 - val_loss: 2.7157 - val_accuracy: 0.1400
Epoch 5/20
7/7 [==============================] - 26s 4s/step - loss: 1.0675 - accuracy: 0.6925 - val_loss: 2.9689 - val_accuracy: 0.1600
Epoch 6/20
7/7 [==============================] - 28s 4s/step - loss: 0.5357 - accuracy: 0.8537 - val_loss: 4.1727 - val_accuracy: 0.1350
Epoch 7/20
7/7 [==============================] - 28s 4s/step - loss: 0.3748 - accuracy: 0.8850 - val_loss: 5.5694 - val_accuracy: 0.2100
Epoch 8/20
7/7 [===============

In [12]:
# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

7/7 - 2s - loss: 5.0957 - accuracy: 0.1000 - 2s/epoch - 292ms/step


('Test accuracy:', 0.10000000149011612)

# LeNet 5 Model

In [ ]:
# Define the LeNet 5 model architecture
model = keras.Sequential([
    layers.Conv2D(6,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(16, kernel_size=(5,5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])